In [283]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
import math
import numpy as np
from sklearn.metrics.pairwise import haversine_distances

In [284]:
path='./data/'

In [310]:
# Считываем файл с последовательным списком остановок на маршрутах ('stop_seq.csv')
stop_seq = pd.read_csv(path + 'stop_seq_patp_for_28.csv', sep = ';', encoding = 'utf8')
stop_seq2 = pd.read_csv(path + 'stop_seq_TB.csv', sep = ';', encoding = 'utf8')

In [311]:
#stop_seq=stop_seq.append(stop_seq2)

In [314]:
stop_seq

,stop_sequence,ssId,ss_lat,ss_long,tt.trip_id,mr_id
0,13,stop__9854819,53.492223,49.301302,2227622945,2227622966
1,14,stop__9854363,53.476237,49.327122,2227622945,2227622966
2,15,stop__9854366,53.475968,49.332976,2227622945,2227622966
3,16,stop__9854368,53.475478,49.344863,2227622945,2227622966
4,21,stop__9854256,53.501115,49.390624,2227622945,2227622966
...,...,...,...,...,...,...
65,37,stop__9854610,53.525948,49.328930,2227622929,2227623008
66,1,3460817685,53.501696,49.273943,2227622962,2227623063
67,33,3453030055,53.542545,49.387210,2227622962,2227623063
68,1,stop__9854562,53.542865,49.387136,2227622965,2227623063


In [315]:
stop_seq_gpd = gpd.GeoDataFrame(stop_seq,  geometry = [Point(xy) for xy in zip(stop_seq['ss_long'], stop_seq['ss_lat'])])
stop_seq_gpd.crs='epsg:4326'

In [316]:
#пересекли все остановки с остановками с буфером 300 метров
stops_with_buff = stop_seq_gpd.to_crs('epsg:32640')
stops_with_buff.geometry = stops_with_buff.geometry.buffer(100)
stops_with_buff = stops_with_buff.to_crs('epsg:4326')
stops_new=gpd.sjoin(stop_seq_gpd, stops_with_buff, how='left', op='within', lsuffix='left', rsuffix='right')

In [317]:
stops_pares=stops_new[stops_new['mr_id_left']==stops_new['mr_id_right']].reset_index()

In [318]:
stops_pares=stops_pares[['ssId_left','ss_lat_left','ss_long_left','ssId_right','ss_lat_right','ss_long_right']].drop_duplicates()

In [205]:
#stops_pares=stops_pares[stops_pares['ssId_left']!=stops_pares['ssId_right']]

In [319]:
stops_pares['geometry']=Point(zip(stops_pares['ss_long_right'], stops_pares['ss_lat_right']))

In [320]:
stops_pares_gpd = gpd.GeoDataFrame(stops_pares,  geometry = 'geometry')
stops_pares_gpd.crs='epsg:4326'

In [321]:
stops_pares_gpd_group=stops_pares_gpd.dissolve(by='ssId_left')

In [322]:
stops_pares

,ssId_left,ss_lat_left,ss_long_left,ssId_right,ss_lat_right,ss_long_right,geometry
0,stop__9854819,53.492223,49.301302,stop__9854819,53.492223,49.301302,POINT (49.30130 53.49222)
1,stop__9854819,53.492223,49.301302,stop__9854818,53.492541,49.300683,POINT (49.30130 53.49222)
2,stop__9854363,53.476237,49.327122,stop__9854363,53.476237,49.327122,POINT (49.30130 53.49222)
3,stop__9854363,53.476237,49.327122,stop__9854820,53.476369,49.326853,POINT (49.30130 53.49222)
4,stop__9854366,53.475968,49.332976,stop__9854366,53.475968,49.332976,POINT (49.30130 53.49222)
...,...,...,...,...,...,...,...
161,3453030055,53.542545,49.387210,stop__9854562,53.542865,49.387136,POINT (49.30130 53.49222)
162,stop__9854562,53.542865,49.387136,3453030055,53.542545,49.387210,POINT (49.30130 53.49222)
163,stop__9854562,53.542865,49.387136,stop__9854562,53.542865,49.387136,POINT (49.30130 53.49222)
164,3460817865,53.501648,49.274131,3460817685,53.501696,49.273943,POINT (49.30130 53.49222)


In [323]:
stops_pares_group=stops_pares.groupby(['ssId_left','ss_lat_left','ss_long_left'])['ssId_right'].apply(list).reset_index()
stops_pares_group2=stops_pares.groupby(['ssId_left','ss_lat_left','ss_long_left'])['ss_lat_right'].apply(list).reset_index()
stops_pares_group3=stops_pares.groupby(['ssId_left','ss_lat_left','ss_long_left'])['ss_long_right'].apply(list).reset_index()

In [324]:
stops_pares_group4=stops_pares.groupby(['ssId_left','ss_lat_left','ss_long_left']).agg({'ss_lat_right': np.mean}).reset_index().rename(columns={'ss_lat_right':'ss_lat'})
stops_pares_group5=stops_pares.groupby(['ssId_left','ss_lat_left','ss_long_left']).agg({'ss_long_right': np.mean}).reset_index().rename(columns={'ss_long_right':'ss_long'})

In [325]:
stops_pares_group=stops_pares_group.merge(stops_pares_group2,on=['ssId_left','ss_lat_left','ss_long_left'],how='left')\
.merge(stops_pares_group3,on=['ssId_left','ss_lat_left','ss_long_left'],how='left')\
.merge(stops_pares_group4,on=['ssId_left','ss_lat_left','ss_long_left'],how='left')\
.merge(stops_pares_group5,on=['ssId_left','ss_lat_left','ss_long_left'],how='left')

In [326]:
sites=pd.DataFrame(stops_pares_group[['ss_lat','ss_long']].drop_duplicates().reset_index())

In [327]:
sites=sites.rename(columns={'index':'site_id'})

In [328]:
stops_pares_group=stops_pares_group.merge(sites,on=['ss_lat','ss_long'],how='left')

In [329]:
stops_pares_group.to_csv('stops_sites_DOP.csv')

In [219]:
len(stops_pares_group)

583

In [242]:
stops_pares_group=pd.read_csv('stops_sites_coordinates.csv')

In [243]:
stops_pares_group=stops_pares_group[['ssId_left','ss_lat_left','ss_long_left','site_id']]

In [244]:
site_group=stops_pares_group.groupby(['site_id']).agg({'ss_lat_left': np.mean,'ss_long_left': np.mean}).reset_index().rename(columns={'ss_lat_left':'ss_lat','ss_long_left':'ss_long'})

In [245]:
site_group['site_id_old']=site_group['site_id']

In [246]:
site_group['site_id']=site_group.index

In [279]:
site_group.to_csv('sites_coordinate.csv', index=False)

In [248]:
stops_pares_group=stops_pares_group.merge(site_group, left_on='site_id', right_on='site_id_old')

In [250]:
stops_pares_group=stops_pares_group[['ssId_left','ss_lat','ss_long','site_id_y']]

In [260]:
stops_pares_group

,ssId_left,ss_lat,ss_long,site_id_y
0,1543173613,53.473269,49.478080,0
1,stop__9854694,53.473269,49.478080,0
2,1543173820,53.577073,49.425409,1
3,1543173824,53.577073,49.425409,1
4,1543174318,53.553636,49.287731,2
...,...,...,...,...
578,stop__9854794,53.539574,49.567475,204
579,stop__9854815,53.498960,49.404614,205
580,stop__9854816,53.498960,49.404614,205
581,stop__9854838,53.479345,49.515677,206


In [273]:
stop_seq=stop_seq.merge(stops_pares_group[['ssId_left','site_id_y']], left_on='ssId',right_on='ssId_left')

In [274]:
stop_seq=stop_seq[['ssId','rc_orderby','ss_lat','ss_long','rl_id','mr_id','site_id_y']]

In [275]:
stop_seq=stop_seq.rename(columns={'site_id_y':'site_id'})

In [281]:
stop_seq=stop_seq.sort_values(by=['mr_id','rl_id','rc_orderby'])

In [282]:
stop_seq.to_csv('stop_seq.csv',index=False)